<DIV ALIGN=CENTER>

# Introduction to Network Analysis: Social Media
## Professor Robert J. Brunner
  
</DIV>  
-----
-----

## Introduction

In this IPython Notebook, we build on 

-----



In [1]:
# Set up Notebook

% matplotlib inline

# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# We do this to ignore several specific Pandas warnings
import warnings
warnings.filterwarnings("ignore")

-----

## Build co-occurance graph


-----

In [ ]:
# Inspired by Fernando Perez's demo code:
# https://gist.github.com/fperez/5681541
import tweepy as tw

# Order: Access Token, Access Token Secret, 
# Consumer Key, Consumer SecretAccess
cred_file = '../../Week8/notebooks/twitter.cred'

tokens = []

with open(cred_file, 'r') as fin:
    for line in fin:
        if line[0] != '#': # Not a comment line
            tokens.append(line.rstrip('\n'))

auth = tw.OAuthHandler(tokens[2], tokens[3])
auth.set_access_token(tokens[0], tokens[1])

api = tw.API(auth, 
             wait_on_rate_limit=True, 
             wait_on_rate_limit_notify=True)

user = api.me()

print("Twitter Screen Name: ", user.screen_name)
print("Twitter Follower Count: ", user.followers_count)

Twitter Screen Name:  ProfBrunner
Twitter Follower Count:  149


In [ ]:
# Number of tweets to get
# Note we will wait until we get them all, so keep small
num_tweets = 2500

# Search term: for us this is hashtag of interest
s_term = '#datascience'

# Lists we use throughout the notebook
users = []
tweets = []

# Get the tweets
results = tw.Cursor(api.search, q=s_term).items(num_tweets)

# Build lists from twitter search results
for result in results:
    users.append(result.user.screen_name)
    tweets.append(result.text)

In [ ]:
print('Total number of tweets = {0}'.format(len(tweets)))

In [ ]:
# Print tweet with most hashtags

cnts = [tweet.count('#') for tweet in tweets]

max_cnt = max(cnts)
max_idx = cnts.index(max_cnt)

print('Max hashtags in tweet (count = {0})'.format(cnts[max_idx]))
print(60*'-')
print(tweets[max_idx])

In [ ]:
# https://docs.python.org/dev/library/collections.html#collections.Counter
# https://docs.python.org/2/library/itertools.html#itertools.combinations
# https://docs.python.org/3/library/string.html#string.punctuation

from collections import Counter
from itertools import combinations
import string

tkns = set()
cnt = Counter()

# Iterate over all tweets
for tweet in tweets:
    
    # First, we use a list comprehension that extract hashtags
    # We strip off leading '#', then remove trailing punctuation.
    # And turn all characters to lowercase.
    # Note this is not Unicode clean.
    # Finally, we sort the list of tags (which will ensure the
    # Counter works correctly)
    
    tags = sorted([tag.lstrip("#").rstrip(string.punctuation).lower() 
            for tag in tweet.split() if tag.startswith("#")])
    
    # We now build our set of tokens
    
    for tag in tags:
        tkns.add(tag)
            
    # Next we use combinations from the itertools library 
    # to make a list of tag pairs. 
    # We then increment the counter for each paried tag

    for paired_tags in combinations(tags, 2):
        cnt[paired_tags] += 1

# How many hashtags did we extract?
print('Total number of unique hashtags = {0}'.format(len(tkns)))

# Extract the top pair, list of tuples (pair, count)
mc = cnt.most_common(1)[0]
print('Total number of paired tags = {0}'.format(len(cnt)))
print('Most common pair = {0}, occurs {1} times.'.format(mc[0], mc[1]))

In [ ]:
# Now we go through pair counter for top pairs to build network.

tkns = set()
num_top_pairs = 25

top_pairs = cnt.most_common(num_top_pairs)

for pair, count in top_pairs:
    tkns.add(pair[0])
    tkns.add(pair[1])

print('Total number of unique hashtags = {0}\n'.format(len(tkns)))
print('Hashtags to graph:')
print(40*'-')
print(tkns)

In [ ]:
# First setup the current figure. 
# No axes, no Axes labels, white background

fig, axes = plt.subplots(figsize=(12, 12))

clrs = sns.color_palette("Paired")

sns.set(style='white', font_scale=1.5)

sns.despine(top = True, bottom = True, 
            left = True, right = True)

axes.set_xticklabels([])
axes.set_yticklabels([])

# Build graph

import networkx as nx

g = nx.Graph()

g.add_nodes_from(list(tkns))

for pair, count in top_pairs:
    g.add_edge(pair[0], pair[1], weight=count)

# Now draw graph, use circular layout

pos = nx.circular_layout(g)

nx.draw_networkx_nodes(g, pos, with_labels=True,
                       node_size = 3000,
                       node_color=clrs[2],
                       node_shape='H',
                       alpha = 0.4)

# Build up list of edges and weights for controlled plotting
edges = []
weights = []

for n1, n2, dct in g.edges(data=True):
    edges.append((n1, n2))
    weights.append(dct['weight'])

# Make normalized weights (scale to minimum weight)
min_weight = min(weights)
norm_weights = [int(weight/min_weight) for weight in weights]

# Draw weighted edges, we pass in the edgelist so we can 
# also pass in the edge weights individually
nx.draw_networkx_edges(g, pos, edgelist=edges, 
                       width = norm_weights, 
                       edge_color=clrs.as_hex()[4],
                       alpha = 0.6)

# Now label nodes, since we don't provide lables, it uses the 
# default labels (which is what we want)
nx.draw_networkx_labels(g, pos, font_size = 18,
                        font_color = clrs[1],
                        alpha = 1.0)

plt.title('Hashtag Co-Occurance Plot for #DataScience')
plt.show()

-----

### Student Activity

In the preceding cells, we introduced ... . Now
that you have run the Notebook, go back and make the following changes
to see how the results change.

1. Change 
2. Replace 
3. Try using 

-----

## Social Graph

-----

In [ ]:
#user = api.me()

# Explicitly get instructor, can change.
user = api.get_user('ProfBrunner')

# We accumulate a list of ids for our friends
my_friends = []

# We map id to screen names to minimize twitter calls
id2name = {}
id2name[user.id] = user.screen_name

# Note: I remove extra users to reduce chance of rate limits later.
# The '*Fact' accounts don't follow peopel (in general)

print('\n{0} follows:\n--------------'.format(user.screen_name))
for friend in user.friends():
    if 'Fact' not in friend.screen_name:
        print(friend.screen_name)
        my_friends.append(friend.id)
        id2name[friend.id] = friend.screen_name

In [ ]:
# To minimize twitter API calls, we grab the friend ids for all of the
# current user's friends. This needs to be less than 15 or we can it
# rate limits. Also, don't rerun frequently, or you hit rate limits.

ids2ids = {}
ids2ids[user.id] = my_friends

# We loop over our friends. We could limit this to first n (n < 15) to
# minimize rate limits

for friend in my_friends:
    
    print('Finding friends for {0}'.format(id2name[friend]))
    
    ids2ids[friend] = api.friends_ids(friend)
    
    print('Found {0} friends.'.format(len(ids2ids[friend])))

In [ ]:
# Now we use the cached friends ids and the 
# dictionary mapping between ids and names
# to build edges in the directed graph.

# Make a copy of the ids, and add the current user id
# Doing this means we can find bi-directional edges
ids = my_friends.copy()
ids.append(user.id)

# Create directed graph and add first node
dg = nx.DiGraph()
dg.add_node(user.screen_name)

# Find edges in social graph and add to the graph.
# We find friends of friends (fof) in the retreived ids
for id in ids:
    for fof in ids2ids[id]:
        if fof in ids:
            dg.add_edge(id2name[id], id2name[fof])

In [ ]:
# First setup the current figure. 
# No axes, no Axes labels, white background

fig, axes = plt.subplots(figsize=(12, 12))

clrs = sns.color_palette("Paired")

sns.set(style='white', font_scale=1.5)

sns.despine(top = True, bottom = True, 
            left = True, right = True)

axes.set_xticklabels([])
axes.set_yticklabels([])

# Now draw graph, use circular layout

#pos = nx.spring_layout(dg)
pos = nx.circular_layout(dg)

nx.draw_networkx_nodes(dg, pos, with_labels=True,
                       node_size = 2000,
                       node_color=clrs[2],
                       node_shape='H',
                       alpha = 0.4)

nx.draw_networkx_edges(dg, pos,
                       width = 1,
                       edge_color=clrs.as_hex()[4],
                       alpha = 0.6)

nx.draw_networkx_labels(dg, pos, font_size = 18,
                        font_color = clrs[1],
                        alpha = 1.0)

plt.title('Social connections for {0}'.format(user.screen_name))
plt.show()

-----

### Student Activity

In the preceding cells, we introduced ... . Now
that you have run the Notebook, go back and make the following changes
to see how the results change.

1. Change 
2. Replace 
3. Try using 

-----

In [ ]:
nx.draw(dg, with_labels=True)

In [ ]:
# First setup the current figure. 
# No axes, no Axes labels, white background

fig, axes = plt.subplots(figsize=(12, 12))

clrs = sns.color_palette("Paired")

sns.set(style='white', font_scale=1.5)

sns.despine(top = True, bottom = True, 
            left = True, right = True)

axes.set_xticklabels([])
axes.set_yticklabels([])

# Now draw graph, use circular layout

pos = nx.spectral_layout(dg)

nx.draw_networkx_nodes(dg, pos, with_labels=True,
                       node_size = 2000,
                       node_color=clrs[2],
                       node_shape='H',
                       alpha = 0.4)

nx.draw_networkx_edges(dg, pos,
                       width = 1,
                       edge_color=clrs.as_hex()[4],
                       alpha = 0.6)

nx.draw_networkx_labels(dg, pos, font_size = 18,
                        font_color = clrs[1],
                        alpha = 1.0)

plt.title('Social connections for {0}'.format(user.screen_name))
plt.show()

In [ ]:
# First setup the current figure. 
# No axes, no Axes labels, white background

fig, axes = plt.subplots(figsize=(12, 12))

clrs = sns.color_palette("Paired")

sns.set(style='white', font_scale=1.5)

sns.despine(top = True, bottom = True, 
            left = True, right = True)

axes.set_xticklabels([])
axes.set_yticklabels([])

# Now draw graph, use circular layout

pos = nx.spring_layout(dg)

nx.draw_networkx_nodes(dg, pos, with_labels=True,
                       node_size = 2000,
                       node_color=clrs[2],
                       node_shape='H',
                       alpha = 0.4)

nx.draw_networkx_edges(dg, pos,
                       width = 1,
                       edge_color=clrs.as_hex()[4],
                       alpha = 0.6)

nx.draw_networkx_labels(dg, pos, font_size = 18,
                        font_color = clrs[1],
                        alpha = 1.0)

plt.title('Social connections for {0}'.format(user.screen_name))
plt.show()